In [18]:
import src.data.loader as loader
import src.models.gps as gps
import torch_geometric.data

# basic test on cora for dev purposes
cora_data, num_classes = loader.load_clean_cora()
model = gps.GPS(cora_data, num_classes, pe_channels=5, hidden_channels=2, observe_attention=True)
w = gps.train(model, cora_data) # w is a list of N attention matrices where N = # of transformer layers
single_matrix = w[0][0]
train_acc, test_acc = gps.test(model, cora_data)
print(f'Train Accuracy for Cora node-level classification: {train_acc}')
print(f'Test Accuracy for Cora node-level classification: {test_acc}')

<class 'torch_geometric.datasets.planetoid.Planetoid'>
torch.Size([2708, 2708])
tensor([ 47,  47,  47,  ...,  47,  47, 130])
Train Accuracy for Cora node-level classification: 1.0
Test Accuracy for Cora node-level classification: 0.185
